In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import geopy.distance



In [2]:
conflict_df = pd.read_csv('../data/conflict_data_irq.csv')
conflict_df.drop(0, inplace=True)
conflict_df['date_start'] = pd.to_datetime(conflict_df['date_start'])
conflict_df['date_end'] = pd.to_datetime(conflict_df['date_start'])

In [3]:
conflict_df.replace(to_replace={'Al Anbār province':'Anbar',
                            'Nīnawá province':'Ninewa',
                           'Baghdād province':'Baghdad', 
                            'Dahūk province':'Dahuk', 
                          'Diyālá province':'Diyala',
                           'Kirkūk province':'Kirkuk',
                           'Şalāḩ ad Dīn province':'Salah al-Din'}, inplace=True)

In [4]:
conflict_df.replace(to_replace={'Abū Ghurayb district':'Abu Ghraib',
'Al Ba‘āj district':"Al-Ba'aj",
'Al Qā’im district':"Al-Ka'im",
'Al-Faris district':'Al-Fares',
'Hīt district':'Heet',
'Qaḑā’ ‘Ānah':'Ana',
'Qaḑā’ ad Dawr':'Al-Daur',
'Qaḑā’ al Fallūjah':'Falluja',
'Qaḑā’ al Ḩaḑr':'Hatra',
'Qaḑā’ al Ḩamdānīyah':'Al-Hamdaniya',
'Qaḑā’ al Khāliş':'Al-Khalis',
'Qaḑā’ al Maḩmūdīyah':'Mahmoudiya',
'Qaḑā’ al Mawşil':'Mosul',
'Qaḑā’ al Miqdādīyah':'Al-Muqdadiya',
'Qaḑā’ ar Ramādī':'Ramadi',
'Qaḑā’ ar Ruţbah':'Al-Rutba',
'Qaḑā’ ash Shaykhān':'Al-Shikhan',
'Qaḑā’ Balad':'Balad',
'Qaḑā’ Bayjī':'Baiji',
'Qaḑā’ Haditha':'Haditha',
'Qaḑā’ Khānaqīn':'Khanaqin',
'Qaḑā’ Kifrī':'Kifri',
'Qaḑā’ Sāmarrā':'Samarra',
'Qaḑā’ Sharqāţ':'Al-Shirqat',
'Qaḑā’ Tall ‘Afar':'Telafar',
'Qaḑā’ Tikrīt':'Tikrit',
'Qaḑā’ Zākhū':'Zakho',
'Sinjār district':'Sinjar',
'Tallkayf district':'Tilkaif',
'Tooz district':'Tuz Khurmatu',
'Zakho district':'Zakho'}, inplace=True)

In [5]:
gov_names = ['Anbar', 'Ninewa', 'Baghdad', 'Dahuk', 'Diyala', 'Salah al-Din']

In [6]:
district_names = ['Abu Ghraib', "Al-Ba'aj", "Al-Ka'im", 'Al-Fares', 'Heet', 'Ana', 'Al-Daur',
'Falluja', 'Hatra', 'Al-Hamdaniya', 'Al-Khalis', 'Mahmoudiya', 'Mosul', 'Al-Muqdadiya', 'Ramadi',
'Al-Rutba', 'Al-Shikhan', 'Balad', 'Baiji', 'Haditha', 'Khanaqin', 'Kifri', 'Samarra', 
'Al-Shirqat', 'Telafar', 'Tikrit', 'Zakho', 'Sinjar', 'Tilkaif', 'Tuz Khurmatu','Zakho']

In [7]:
conflict_df = conflict_df[conflict_df['adm_1'].isin(gov_names)]
conflict_df = conflict_df[conflict_df['adm_2'].isin(district_names)]

In [8]:
%%time
outflow_filepaths = [s for s in listdir("../data/out/")]
out_df = pd.concat((pd.read_csv("../data/out/"+s) for s in outflow_filepaths), ignore_index=True) 


CPU times: user 4.23 s, sys: 969 ms, total: 5.2 s
Wall time: 6.14 s


In [9]:
out_df['date'] = pd.to_datetime(out_df['date'])
out_df.rename(columns={'Location Name':'Location_name'})
out_df = out_df[out_df['District'].isin(district_names)]

In [10]:
%%time
returnee_filepaths = [f for f in listdir("../data/returnees/") if f.endswith('.xlsx')]
ret_df = pd.concat((pd.read_excel("../data/returnees/"+f).assign(date=f[35:47])
                  for f in returnee_filepaths), ignore_index=True)

CPU times: user 2min 7s, sys: 2.11 s, total: 2min 9s
Wall time: 2min 16s


In [11]:
ret_df.rename(columns={'Pre June14 Period of displacement':'disp_preJun14',
                      'June July14 Period of displacement':'disp_JunJuly14',
                      'August14 Period of displacement':'disp_Aug14',
                      'Post September 14 Period of displacement':'disp_postSep14',
                      'Post April15 Period of displacement':'disp_postApr15',
                      'Post March 16 Period of displacement':'disp_postMar16',
                      'Post 17 October 16 Period of displacement': 'disp_post17Oct16',
                      'July 17 Period of displacement':'disp_Jul17',
                      'Jan19':'disp_Jan19'}, inplace=True)

In [12]:
ret_df.dropna(how='all', inplace=True)
ret_df['date'] = pd.to_datetime(ret_df['date'])

In [13]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)



In [14]:
ret_df = trim_all_columns(ret_df)
out_df = trim_all_columns(out_df)

In [15]:
ret_df.sort_values(['Location_name', 'date'], inplace=True)

In [16]:
ret_df.dropna(how='any', inplace=True)

In [17]:
ret_df['ret_delta'] = ret_df.groupby(['Location_name'])['Returnee Families'].transform(lambda x: x.diff()) 



In [18]:
ret_df.sort_values(['Location_name', 'date'], inplace=True)
ret_df.reset_index(inplace=True)
ret_df.drop(columns='index', axis=1, inplace=True)

In [19]:
#should be functionized 
for i in range(len(ret_df)):
    if np.isnan(ret_df.at[i, 'ret_delta']):
        ret_df.at[i, 'ret_delta'] = ret_df.at[i, 'Returnee Families']
    else:
        None

In [26]:
out_df['out_delta'] = out_df.groupby(['Location Name'])['Families'].transform(lambda x: x.diff()) 


In [41]:
for i in range(len(out_df)):
    if np.isnan(out_df.at[i, 'out_delta']):
        out_df.at[i, 'out_delta'] = out_df.at[i, 'Families']
    else:
        None

KeyError: 0

In [ ]:
ser = pd.Series(index=ret_df.index, data=0)

In [ ]:
ret_df['District'].unique()

In [ ]:
print(7473*26000)

In [ ]:
for ix, coord in enumerate(zip(conflict_df['latitude'], conflict_df['longitude'])):
    for idx, rcoord in enumerate(zip(ret_df['Latitude'], ret_df['Longitude'])):
        if geopy.distance.distance(coord, rcoord).km > 10:
            print(idx)
            ser[idx] += 1
        else:
            print(idx)